In [1]:
from sklearn.datasets import *
import gc
from LightGBM.LightGBM_ToolBox_ver2 import *

%load_ext autoreload
%autoreload 2

c:\users\xiaowei.li\appdata\local\continuum\anaconda3\envs\myvirtualpythonenv\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Load data:

In [2]:
t = load_boston()

data = pd.DataFrame(load_boston().data)
data.columns = t.feature_names
target = load_boston().target

In [3]:
print(data.shape)
data.head()

(506, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [4]:
random_idx = np.random.choice(data.index, size=int(data.shape[0]*0.2))
print(len(random_idx))

101


In [5]:
print('Change partial data to NULL...')
data.loc[random_idx, 'AGE'] = np.nan
gc.collect()

Change partial data to NULL...


0

In [6]:
print(np.sort(data['AGE'].unique()))

[  2.9   6.    6.2   6.5   6.6   6.8   7.8   8.9   9.9  10.   13.9  14.7
  15.3  15.7  15.8  16.3  17.   17.2  17.5  17.7  17.8  18.4  18.5  18.8
  19.1  19.5  20.1  20.8  21.1  21.4  21.5  21.8  21.9  22.3  22.9  23.3
  24.8  25.8  26.3  27.7  27.9  28.1  28.4  28.8  28.9  29.1  29.2  29.3
  30.2  30.8  31.1  31.3  31.5  31.9  32.   32.1  32.2  32.3  32.9  33.
  33.1  33.2  33.3  33.5  34.1  34.5  35.9  36.1  36.6  36.8  37.2  37.3
  37.8  38.1  38.3  38.4  38.5  38.9  39.   40.   40.1  40.3  40.5  41.1
  41.5  41.9  42.2  42.3  42.4  43.4  43.7  44.4  45.   45.4  45.6  45.7
  45.8  46.7  47.2  47.6  48.   48.2  49.   49.1  49.3  49.7  49.9  51.
  51.8  51.9  52.3  52.5  52.6  52.8  53.2  53.6  53.7  53.8  54.   54.2
  54.3  54.4  56.   56.1  56.4  56.5  56.7  56.8  58.   58.1  58.4  58.7
  58.8  59.5  59.6  59.7  61.1  61.4  62.   62.2  62.5  62.8  63.   63.1
  64.7  65.1  65.2  65.3  65.4  66.5  66.6  67.   67.2  67.6  67.8  68.1
  68.7  68.8  69.1  69.5  69.6  70.2  70.3  70.4  70.

### Test modules:

In [7]:
mylight = MyLight(objective='regression')

In [8]:
mylight.get_log()

[01/24/2019 03:51:58 PM (Local Time)] INFO : Start logging...
[01/24/2019 03:51:58 PM (Local Time)] INFO : Update instance booster params to: {'boosting_type': 'gbdt', 'subsample': 1.0, 'min_child_samples': 20, 'n_estimators': 100, 'subsample_freq': 0, 'max_depth': -1, 'objective': 'regression', 'class_weight': None, 'reg_lambda': 0.0, 'reg_alpha': 0.0, 'min_child_weight': 0.001, 'colsample_bytree': 1.0, 'min_split_gain': 0.0, 'subsample_for_bin': 200000, 'num_leaves': 31}



In [9]:
print(mylight.lgb_default_metrics)
print(mylight.lgb_default_objectives)

['l1', 'mean_absolute_error', 'mae', 'regression_l1', 'l2', 'mean_squared_error', 'mse', 'regression_l2', 'regression', 'l2_root', 'root_mean_squared_error', 'rmse', 'quantile', 'mape', 'mean_absolute_percentage_error', 'huber', 'fair', 'poisson', 'gamma', 'gamma_deviance', 'tweedie', 'ndcg', 'map', 'auc', 'binary_logloss', 'binary_error', 'multi_logloss', 'multiclass', 'softmax', 'multiclassova', 'multiclass_ova', 'ova', 'ovr', 'multi_error', 'xentropy', 'cross_entropy', 'xentlambda', 'cross_entropy_lambda', 'kldiv', 'kullback_leibler']
['regression', 'regression_l1', 'huber', 'fair', 'poisson', 'quantile', 'mape', 'gamma', 'tweedie', 'binary', 'multiclass', 'multiclassova', 'cross_entropy', 'cross_entropy_lambda', 'lambdarank']


In [10]:
n_rounds = 500
learning_rates=[0.1, 0.001, 0.0001]
for i in range(n_rounds-3):
    learning_rates.append(0.00001)

In [11]:
model = mylight.fit(data, target, learning_rates=learning_rates, verbose_eval=100, eval_metric=['rmse'], num_iterations=n_rounds, n_jobs=3)

model training
Training until validation scores don't improve for 100 rounds.
[100]	Training's rmse: 8.62894	Validation's rmse: 7.91659
[200]	Training's rmse: 8.62196	Validation's rmse: 7.91088
[300]	Training's rmse: 8.61497	Validation's rmse: 7.90517


c:\users\xiaowei.li\appdata\local\continuum\anaconda3\envs\myvirtualpythonenv\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[400]	Training's rmse: 8.60801	Validation's rmse: 7.89949
[500]	Training's rmse: 8.60106	Validation's rmse: 7.89385
Did not meet early stopping. Best iteration is:
[500]	Training's rmse: 8.60106	Validation's rmse: 7.89385
*********************************************
For model training, Time cost: 0.35 second(s)
*********************************************


In [12]:
mylight.analyze_feature_importance(plot=False)

,Features,Importances
0,CRIM,502
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,500
5,RM,1999
6,AGE,0
7,DIS,0
8,RAD,1
9,TAX,499


In [13]:
trained_models = mylight.kfolds_cv_ensemble(data, target, learning_rate=0.0001, num_iterations=100)

Start K-Fold cross validation (ensemble)...

For fold 1...

model training
Training until validation scores don't improve for 100 rounds.
[100]	Training's rmse: 9.3296	Validation's rmse: 8.67506
Did not meet early stopping. Best iteration is:
[100]	Training's rmse: 9.3296	Validation's rmse: 8.67506
*********************************************
For model training, Time cost: 0.09 second(s)
*********************************************

For fold 2...

model training
Training until validation scores don't improve for 100 rounds.
[100]	Training's rmse: 9.0346	Validation's rmse: 9.28066
Did not meet early stopping. Best iteration is:
[100]	Training's rmse: 9.0346	Validation's rmse: 9.28066
*********************************************
For model training, Time cost: 0.09 second(s)
*********************************************

For fold 3...



c:\users\xiaowei.li\appdata\local\continuum\anaconda3\envs\myvirtualpythonenv\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


model training
Training until validation scores don't improve for 100 rounds.
[100]	Training's rmse: 8.973	Validation's rmse: 9.39669
Did not meet early stopping. Best iteration is:
[100]	Training's rmse: 8.973	Validation's rmse: 9.39669
*********************************************
For model training, Time cost: 0.08 second(s)
*********************************************
For 3-folds cross validation, the rmse is 9.11608


In [14]:
mylight.get_log()

[01/24/2019 03:51:58 PM (Local Time)] INFO : Start logging...
[01/24/2019 03:51:58 PM (Local Time)] INFO : Update instance booster params to: {'boosting_type': 'gbdt', 'subsample': 1.0, 'min_child_samples': 20, 'n_estimators': 100, 'subsample_freq': 0, 'max_depth': -1, 'objective': 'regression', 'class_weight': None, 'reg_lambda': 0.0, 'reg_alpha': 0.0, 'min_child_weight': 0.001, 'colsample_bytree': 1.0, 'min_split_gain': 0.0, 'subsample_for_bin': 200000, 'num_leaves': 31}
[01/24/2019 03:51:59 PM (Local Time)] INFO : Randomly split training data into 70% to 30%.
[01/24/2019 03:51:59 PM (Local Time)] INFO : Training data size: (354, 13), validation data size: (152, 13)
[01/24/2019 03:51:59 PM (Local Time)] INFO : Booster parameters: {'seed': 7, 'boosting_type': 'gbdt', 'subsample': 1.0, 'metrics': ['rmse'], 'min_child_samples': 20, 'n_estimators': 100, 'num_iterations': 500, 'subsample_freq': 0, 'max_depth': -1, 'objective': 'regression', 'class_weight': None, 'n_jobs': 3, 'reg_lambda':